<a href="https://colab.research.google.com/github/richardGonzalez-std/InventarioKivyMVC/blob/main/tools/Build_APK_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SIAM - Compilar APK en Google Colab

**Tiempo estimado:** ~30 min

---

## 1. Configuración

In [ ]:
# ============================================
# EDITA ESTAS LINEAS
# ============================================
GITHUB_REPO = "https://github.com/richardGonzalez-std/InventarioKivyMVC.git"
BRANCH = "main"

print(f"Repo: {GITHUB_REPO}")
print(f"Rama: {BRANCH}")

## 2. Montar Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print('Drive montado')

## 3. Clonar repositorio

In [ ]:
import os
import shutil
from pathlib import Path

WORK_DIR = '/content/InventarioKivyMVC'

# Asegurar directorio base seguro
os.chdir('/content')

# Limpiar directorio de trabajo
if os.path.exists(WORK_DIR):
    shutil.rmtree(WORK_DIR)

# Clonar repo
!git clone --branch {BRANCH} --depth 1 {GITHUB_REPO} {WORK_DIR}

os.chdir(WORK_DIR)
print(f'\nProyecto clonado en: {WORK_DIR}')

## 4. Instalar dependencias

In [ ]:
print('Instalando dependencias del sistema...')
!apt-get update -qq
!apt-get install -y -qq \
    python3-pip build-essential git python3-dev \
    ffmpeg libsdl2-dev libsdl2-image-dev libsdl2-mixer-dev libsdl2-ttf-dev \
    libportmidi-dev libswscale-dev libavformat-dev libavcodec-dev zlib1g-dev \
    libgstreamer1.0-dev gstreamer1.0-plugins-base gstreamer1.0-plugins-good \
    zip unzip openjdk-17-jdk autoconf libtool pkg-config \
    libncurses5-dev libncursesw5-dev cmake libffi-dev libssl-dev \
    gettext autopoint libltdl-dev libtool-bin lld m4
!apt-get install -y -qq \
    libsqlite3-dev sqlite3 bzip2 libbz2-dev openssl \
    libgdbm-dev libgdbm-compat-dev liblzma-dev lzma \
    uuid-dev libreadline-dev libdb-dev expect

!pip install --upgrade pip -q
!pip install --upgrade buildozer cython==0.29.36 -q
print('Dependencias instaladas')

## 5. Compilar APK

In [ ]:
import time

os.chdir(WORK_DIR)

start = time.time()
!buildozer android clean
!buildozer -v android debug 2>&1 | tee build.log
elapsed = time.time() - start

print(f'\nTiempo de compilacion: {elapsed/60:.1f} minutos')

## 6. Guardar APK en Drive

In [ ]:
bin_path = Path(WORK_DIR) / 'bin'
apks = list(bin_path.glob('*.apk')) if bin_path.exists() else []

if apks:
    apk_path = apks[0]
    size_mb = apk_path.stat().st_size / 1024 / 1024
    print(f'APK: {apk_path.name}')
    print(f'Tamano: {size_mb:.1f} MB')

    output_dir = '/content/drive/MyDrive/APKs'
    os.makedirs(output_dir, exist_ok=True)
    dest = f'{output_dir}/{apk_path.name}'
    shutil.copy2(apk_path, dest)
    print(f'\nGuardado en: {dest}')
else:
    print('ERROR: No se genero el APK')
    print('\nErrores encontrados:')
    !grep -i "error" build.log | tail -20

## 7. Descargar APK

In [ ]:
if apks:
    from google.colab import files
    files.download(str(apk_path))